# Revista de Informação Contábil – RIC

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(query, url_search, pg_num=1):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl_results = soup.find('table', {'class': 'listing'})
    a_links = []
    for a in tbl_results.find_all('a'):
           if 'Resumo' in a.text:
                a_links.append(a['href'])
    return a_links



In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(query, url_search, pg_num)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links



In [5]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [6]:
def captura_titulo(soup):
    try:
        title_tag = soup.find('div', {'id': 'articleTitle'})
        title = title_tag.find('h3').text
    except:
        title = ''
    return {'Título': title}

In [7]:
def captura_ano(soup):
    try:
        year_tags = soup.find('div', {'id': 'breadcrumb'})
        year_tag = year_tags.find_all('a')
        year_text = year_tag[1].text
        year = year_text[-5:-1]
    except:
        year = ''
    return {'Data de Publicação': year}

In [8]:
def captura_resumo(soup):
    try:
        abstract_tag = soup.find('div', {'id': 'articleAbstract'})
        abstract = abstract_tag.find('div').text
    except:
        abstract = ''
    return {'Resumo': abstract}

In [9]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('div', {'id': 'articleSubject'})
        kw = kw_tag.find('div').text.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [10]:
def captura_autores(soup):
    autores = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        
        for p in authors_tag:
            try:
                autor = p.find('em').text
            except:
                autor = ''
            autores.append(autor)
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [11]:
def captura_afiliacao(soup):
    affiliation_list = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        for p in authors_tag:
            try:
                affiliation = p.find_all('br')[-2].next_sibling
                affiliation = affiliation.replace('\t', '')
            except:
                affiliation = ''
            affiliation_list.append(affiliation)
    except:
        pass
    affiliation_list = completar_lista(affiliation_list)
    key_affiliation = [f"Afilição {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, affiliation_list))

In [12]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Informação Contábil – RIC'}
    url_search_text = 'https://periodicos.ufpe.br/revistas/ricontabeis/search/search?query=__query__&searchJournal=37&authors=&title=&abstract=&galleyFullText=&suppFiles=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=01&dateFromDay=1&dateFromYear=2012&dateToMonth=12&dateToDay=31&dateToYear=2021&orderBy=&orderDir=&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_ano(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y')
    return df

## Pesquisa por artigos

In [13]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2018-01-01,UMA ANÁLISE ACERCA DAS MUDANÇAS NA CONTABILIZA...,Levando em consideração as alterações do métod...,Ativos Biológicos. Ativos Biológicos Portadore...,3,Arthur Frederico Lerner,Vanessa Noguez Machado,Romina Batista de Lucena de Souza,,,,Universidade Federal do Rio Grande do Sul (UFRGS),Universidade Federal do Rio Grande do Sul (UFRGS),Universidade Federal do Rio Grande do Sul (UFRGS),,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
1,2018-01-01,NÍVEL DE EVIDENCIAÇÃO AMBIENTAL DAS COMPANHIAS...,O presente estudo tem como objetivo verificar ...,NBC T-15. Setor siderúrgico. Evidenciação Ambi...,3,Marcos Antônio Ferreira do Patrocínio,Alex Martins Moura,Adriana Rodrigues Silva,,,,,,Universidade Federal do Pará,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
2,2018-01-01,RECONHECIMENTO E EVIDENCIAÇÃO DE RECEITAS: O ...,Esse estudo teve como objetivo buscar o entend...,Organizações Sociais. Receitas Operacionais. R...,3,Ana Beatriz Gonçalves de Carvalho,Abimael de Jesus Barros Costa,Bruno Vinícius Ramos Fernandes,,,,,Universidade de Brasília,UnB,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC


In [14]:
df.to_csv('../data/RIC.csv', index=False)

In [15]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2018-01-01,UMA ANÁLISE ACERCA DAS MUDANÇAS NA CONTABILIZA...,Levando em consideração as alterações do métod...,Ativos Biológicos. Ativos Biológicos Portadore...,3,Arthur Frederico Lerner,Vanessa Noguez Machado,Romina Batista de Lucena de Souza,,,,Universidade Federal do Rio Grande do Sul (UFRGS),Universidade Federal do Rio Grande do Sul (UFRGS),Universidade Federal do Rio Grande do Sul (UFRGS),,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
1,2018-01-01,NÍVEL DE EVIDENCIAÇÃO AMBIENTAL DAS COMPANHIAS...,O presente estudo tem como objetivo verificar ...,NBC T-15. Setor siderúrgico. Evidenciação Ambi...,3,Marcos Antônio Ferreira do Patrocínio,Alex Martins Moura,Adriana Rodrigues Silva,,,,,,Universidade Federal do Pará,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
2,2018-01-01,RECONHECIMENTO E EVIDENCIAÇÃO DE RECEITAS: O ...,Esse estudo teve como objetivo buscar o entend...,Organizações Sociais. Receitas Operacionais. R...,3,Ana Beatriz Gonçalves de Carvalho,Abimael de Jesus Barros Costa,Bruno Vinícius Ramos Fernandes,,,,,Universidade de Brasília,UnB,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
3,2012-01-01,ANÁLISE DE SUSTENTABILIDADE AMBIENTAL: ESTUDO ...,Considerando a importância da sustentabilidade...,Responsabilidade Social. Sustentabilidade e Es...,3,Eduardo Coelho,Daniela Di Domênico,Elisete Dahmer Pfitscher,,,,UFSC,FURB,UFSC,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
4,2016-01-01,INDICADORES DE DESEMPENHO AMBIENTAL: UM ESTUDO...,O objetivo do estudo foi analisar a produção d...,Indicadores de desempenho. Indicadores ambient...,3,DANIELA FAGUNDES CARNELÓS NUNES,EMANUELLI MARIANA MAINGUÉ,EDWIN VLADIMIR CARDOZA GALDAMEZ,,,,Universidade Estadual de Maringá,,,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
5,2012-01-01,PLANEJAMENTO E CONTROLE DE CUSTOS AMBIENTAIS: ...,O objetivo do artigo é a investigação da práti...,Gestão Ambiental. Custos Ambientais. Planejame...,3,Marcos Antonio Souza,Ana Cristine Heinen,Franciele Mezzomo,,,,UNISINOS,Unisinos,Unisinos,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
6,2015-01-01,SUSTENTABILIDADE AMBIENTAL: UMA ANÁLISE COMPAR...,O objetivo deste estudo é analisar a sustentab...,,3,Samanta Martins Saldano,Vivian Osmari Uhlmann,Elisete Dahmer Pfitscher,,,,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,Universidade Federal de Santa Catarina,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
7,2014-01-01,GESTÃO ESTRATÉGICA BASEADA NOS CUSTOS DA QUALI...,O objetivo deste estudo consiste em identifica...,Estratégia. Custos da qualidade. Construção ci...,3,Janayna Rodrigues de Morais Luz,Paulo Roberto da Nóbrega Cavalcante,José Ribamar Marques Carvalho,,,,Unesc Faculdades e Favip,Universidade Federal da Paraíba,Universidade Federal de Campina Grande,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
8,2015-01-01,UMA ANÁLISE DOS PROGRAMAS DE INCENTIVO A CIDAD...,O objetivo do estudo é analisar os Programas d...,Programas de Estímulos. Cidadania Fiscal. Ince...,4,Nicole Stefani Campos Ramos,Luiz Felipe Ferreira,Sérgio Murilo Petri,Priscila Alano da Rosa,,,Universidade Federal de Santa Catarina,,,,,,https://periodicos.ufpe.br/revistas/ricontabei...,Revista de Informação Contábil – RIC
9,2020-01-01,Ações de Práticas Ambientais no Planejamento d...,A pesquisa teve por objetivo investigar a real...,Auditoria Interna. Ações de Práticas Ambientai...,5,Cristiane Alves da Silva Moura,Edicreia Andrade dos Santos,Pedro Henrique Da Silva Melo Pereira,A